In [1]:
import yaml
import scipy.io as sio
import numpy as  np
from skimage.data import shepp_logan_phantom

dict_keys(['__header__', '__version__', '__globals__', 'b1', 'kdata', 'k', 'w'])


In [2]:
# load the data
with open("../data_loc.yaml", "r") as f:
    data_file = yaml.safe_load(f)

raw_data = sio.loadmat(data_file)

print(raw_data.keys())

# create a simple "fake" image to play round with
xtrue = shepp_logan_phantom()
xtrue = xtrue[8:-8, 8:-8]
sensitivity_maps = np.transpose(raw_data["b1"], (2, 0, 1))

dict_keys(['__header__', '__version__', '__globals__', 'b1', 'kdata', 'k', 'w'])


In [3]:
print(f"image shape: {xtrue.shape}, sense maps shape: {sensitivity_maps.shape}")

# simplest SENSE implementation
output = []
for sensitivity_map in sensitivity_maps:
    output.append(xtrue * sensitivity_map)

output = np.stack(output)
print(f"output shape: {output.shape}")

image shape: (384, 384), sense maps shape: (12, 384, 384)
output shape: (12, 384, 384)


In [4]:
# broadcasting demonstration
# these three operations are equivalent!

In [5]:
# operation 1, slowest
output = []
for sensitivity_map in sensitivity_maps:
    output.append(xtrue * sensitivity_map)

output1 = np.stack(output)
print(
    f"A shape: {sensitivity_maps.shape}, B shape: {xtrue.shape}, "
    f"output shape: {output1.shape}"
)

A shape: (12, 384, 384), B shape: (384, 384), output shape: (12, 384, 384)


In [6]:
# operation 2, better
xtrue_expand = xtrue[None, ...]
xtrue_copy = np.repeat(xtrue_expand, 12, axis=0)
output2 = sensitivity_maps * xtrue_copy
print(
    f"A shape: {sensitivity_maps.shape}, B shape: {xtrue_copy.shape}, "
    f"output shape: {output2.shape}"
)

A shape: (12, 384, 384), B shape: (12, 384, 384), output shape: (12, 384, 384)


In [7]:
# operation 3, fasteset
output3 = sensitivity_maps * xtrue_expand
print(
    f"A shape: {sensitivity_maps.shape}, B shape: {xtrue_expand.shape}, "
    f"output shape: {output3.shape}"
)

A shape: (12, 384, 384), B shape: (1, 384, 384), output shape: (12, 384, 384)


In [8]:
# test they're equal!
print((output1 == output2).all())
print((output1 == output3).all())

True
True


In [9]:
# speed demo
import time

def op1(sensitivity_maps, xtrue):
    output = []
    for sensitivity_map in sensitivity_maps:
        output.append(xtrue * sensitivity_map)

    return np.stack(output)

def op2(sensitivity_maps, xtrue):
    xtrue_expand = xtrue[None, ...]
    xtrue_copy = np.repeat(xtrue_expand, 12, axis=0)
    return sensitivity_maps * xtrue_copy

def op3(sensitivity_maps, xtrue):
    xtrue_expand = xtrue[None, ...]
    return sensitivity_maps * xtrue_expand

num_tests = 1000
op_speeds = {}
for ind, op in zip(range(1, 4), [op1, op2, op3]):
    start_time = time.perf_counter()
    for _ in range(num_tests):
        output = op(sensitivity_maps, xtrue)
    end_time = time.perf_counter()

    op_speed = (end_time - start_time) / num_tests
    print(f"op{ind} speed: {op_speed} seconds")

op1 speed: 0.006369061792007415 seconds
op2 speed: 0.00525306937500136 seconds
op3 speed: 0.004670845333006582 seconds


In [13]:
# build the adjoint operation
def op_adjoint(sensitivity_maps, fy):
    return np.sum(np.conj(sensitivity_maps) * fy, axis=0)

output = op_adjoint(sensitivity_maps, output)

# test the adjoint operation
im_shape = (1, xtrue.shape[-2], xtrue.shape[-1])
coil_im_shape = (sensitivity_maps.shape[0], xtrue.shape[-2], xtrue.shape[-1])
vec1 = np.random.normal(size=im_shape) + 1j*np.random.normal(size=im_shape)
vec2 = np.random.normal(size=coil_im_shape) + 1j*np.random.normal(size=coil_im_shape)

def complex_tensor_inprod(a, b):
    return np.sum(np.conj(a) * b)

inprod1 = complex_tensor_inprod(op3(sensitivity_maps, vec1), vec2)
inprod2 = complex_tensor_inprod(vec1, op_adjoint(sensitivity_maps, vec2))

print(np.allclose(inprod1, inprod2))

True
